In [12]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var 


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [16]:
#### 데이터 불러오기

T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

In [17]:
#### Part II : Writing the Network
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16,self).__init__()
    self.cnn1 = nn.Conv2d(1,3,3)
    self.cnn2 = nn.Conv2d(3,2,5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x

In [ ]:
#### Part III : Writing the main Training loop

mycnn = VGG16().to(device)
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr = learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100.
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.to(device))
    x = model(images)
    value,pred = torch.max(x,1)
    pred = pred.data.cpu()
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100./total

for e in range(n_epoch):
  for i,(images,labels) in enumerate(train_dl):
    images = var(images.to(device))
    labels = var(labels.to(device))
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

Epoch : 1 Batch : 10 Loss : 2.1758499145507812 Accuracy : 35.95000076293945 %
Epoch : 1 Batch : 20 Loss : 1.3741941452026367 Accuracy : 68.48999786376953 %
Epoch : 1 Batch : 30 Loss : 0.6325315833091736 Accuracy : 77.0999984741211 %
Epoch : 1 Batch : 40 Loss : 0.45463693141937256 Accuracy : 80.22000122070312 %
Epoch : 1 Batch : 50 Loss : 0.45747122168540955 Accuracy : 82.76000213623047 %
Epoch : 1 Batch : 60 Loss : 0.6971485614776611 Accuracy : 84.0999984741211 %
Epoch : 1 Batch : 70 Loss : 0.3987387716770172 Accuracy : 84.43000030517578 %
Epoch : 1 Batch : 80 Loss : 0.42176491022109985 Accuracy : 85.30000305175781 %
Epoch : 1 Batch : 90 Loss : 0.40984469652175903 Accuracy : 85.44000244140625 %
Epoch : 1 Batch : 100 Loss : 0.2697928249835968 Accuracy : 86.29000091552734 %
Epoch : 1 Batch : 110 Loss : 0.6030862331390381 Accuracy : 86.30999755859375 %
Epoch : 1 Batch : 120 Loss : 0.4839753806591034 Accuracy : 86.69000244140625 %
Epoch : 1 Batch : 130 Loss : 0.4545327425003052 Accuracy : 